<a href="https://colab.research.google.com/github/jzfrank/h4g-idmc-articleClassifier/blob/main/idmc_article_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.5 MB/s 
     |████████████████████████████████| 6.6 MB 38.6 MB/s 
     |████████████████████████████████| 163 kB 57.6 MB/s 


In [10]:
import torch

In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("sacculifer/dimbat_disaster_distilbert")
model = AutoModelForSequenceClassification.from_pretrained("sacculifer/dimbat_disaster_distilbert", from_tf=True)

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [45]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}

In [8]:
text = "This model was trained from part of Disaster Tweet Corpus 2020 (Analysis of Filtering Models for Disaster-Related Tweets, Wiegmann,M. et al, 2020) dataset It achieves the following results on the evaluation set"
inputs = tokenizer(text, return_tensors="pt")


In [11]:
with torch.no_grad():
    logits = model(**inputs).logits

In [13]:
logits

tensor([[-1.2735,  1.6229]])

In [14]:

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_1'

In [22]:
# diaster type classification

tokenizer = AutoTokenizer.from_pretrained("sacculifer/dimbat_disaster_type_distilbert")
model = AutoModelForSequenceClassification.from_pretrained("sacculifer/dimbat_disaster_type_distilbert", from_tf=True)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [26]:
text = "ABC: About 30 more homes evacuated in Arizona brush fire"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
  logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()

In [28]:
type(predicted_class_id)

int

In [25]:
text

'This model was trained from part of Disaster Tweet Corpus 2020 (Analysis of Filtering Models for Disaster-Related Tweets, Wiegmann,M. et al, 2020) dataset It achieves the following results on the evaluation set'

Definig Classes

In [47]:
class IsDisasterClassifier:
  def __init__(self):
    self.tokenizer = AutoTokenizer.from_pretrained("sacculifer/dimbat_disaster_distilbert")
    self.model = AutoModelForSequenceClassification.from_pretrained("sacculifer/dimbat_disaster_distilbert", from_tf=True)
  def isDisaster(self, text):
    inputs = self.tokenizer(text, return_tensors="pt")
    with torch.no_grad():
      logits = self.model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return {
        1: True, 
        0: False
    }[
        predicted_class_id
    ]

In [48]:
class DisasterTypeClassifier:
  def __init__(self):
    self.tokenizer = AutoTokenizer.from_pretrained("sacculifer/dimbat_disaster_type_distilbert")
    self.model = AutoModelForSequenceClassification.from_pretrained("sacculifer/dimbat_disaster_type_distilbert", from_tf=True)
  def disasterType(self, text):
    inputs = self.tokenizer(text, return_tensors="pt")
    with torch.no_grad():
      logits = self.model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return {
        1: "disease",
        2: "earthquake",
        3: "flood",
        4: "hurricane & tornado",
        5: "wildfire",
        6: "industrial accident",
        7: "societal crime",
        8: "transportation accident",
        9: "meteor crash",
        0: "haze"
    }[
        predicted_class_id
    ]


In [49]:
class ArticleClassifier:
  def __init__(self):
    self.isDisasterClassifier = IsDisasterClassifier()
    self.disasterTypeClassifier = DisasterTypeClassifier()
  def isDisaster(self, text: str) -> bool:
    return self.isDisasterClassifier.isDisaster(text)
  def disasterType(self, text: str) -> str:
    if not self.isDisaster(text):
      return "not a disaster"
    return self.disasterTypeClassifier.disasterType(text)

In [50]:
ac = ArticleClassifier()

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.
All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [2]:
examples = [
    "NBC: Evacuations Lifted in 1,100-Acre Brush Fire in Santa Clarita Valley",
    "KRQE News: Dog Head Fire: Information for evacuees",
    "France 24: Hurricane Fiona batters Turks and Caicos after devastating Puerto Rico - 21/09/2022",
    "Russia-Ukraine War Explosion Damages Crimea Bridge, Imperiling Russian Supply Route",
    "Arizona court halts enforcement of near-total abortion ban",
    "Wow, Google Really, Really Wants to Be Cooler Than Apple",
    "The Hack4Good coordinator is in charge of facilitating the communication between the H4G Organization Committee to address any organizational issue which might arise."
]

In [3]:
for example in examples:
  isDisaster = ac.isDisaster(example)
  disasterType = ac.disasterType(example)
  print(f"{example} \n isDisaster? {isDisaster}\n disasterType? {disasterType} \n\n")

NameError: ignored